In [1]:
# Trainer로 MLM 훈련시킨 모델에 대해 평가해보는 예시 

import torch
import os

from tqdm.notebook import tqdm
from transformers import AutoTokenizer, DistilBertForMaskedLM,BertForMaskedLM, RobertaForMaskedLM
from transformers import AdamW, get_linear_schedule_with_warmup

import sys
sys.path.append("..")
from myutils import GPU_info, seed_everything, mlogging

# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

In [2]:
# eval 말뭉치 
is_Localfile = False   # 로컬파일이면 = True, 허깅페이스면 False
eval_corpus = "bongsoo/bongeval"
#eval_corpus = "bongsoo/moco_eval"
#eval_corpus = "../../data11/my_corpus/klue_dp_valid_text.txt"

# 기존 사전훈련된 모델
# model 타입 : 0=distilbert, 1=bert, 2=Roberta
#=>Roberta 모델에는 distilbert처럼 token_type_id 입력 없음.
model_type = 0
model_path = "../../data11/model/distilbert/bert-re-kowiki-mecab/"
#model_path = "bert-base-multilingual-cased"

# 출력
OUTPATH = '../../data11/model/distilbert/bert-re-kowiki-morphs-out/'

############################################################################
# tokenizer 관련 hyper parameter 설정
############################################################################
batch_size = 32       # batch_size
token_max_len = 128   # token_seq_len
############################################################################

device = GPU_info()
print(device)

#seed 설정
seed_everything(333)

#logging 설정
logger =  mlogging(loggername="distilbert-MLM-Test-Trainer", logfilename="../../log/distilbert-MLM-Test-Trainer")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:../../log/distilbert-MLM-Test-Trainer_2022-10-20.log


In [3]:
# tokeinzier 생성
tokenizer = AutoTokenizer.from_pretrained(model_path, max_len=token_max_len, do_lower_case=False)

# fast 토크너나이즈인지 확인
print(f'{model_path} is_fast:{tokenizer.is_fast}')

# 모델 로딩 further pre-training 
if model_type == 0:
    model = DistilBertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 
elif model_type == 1:
    model = BertForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 
elif model_type == 2:
    model = RobertaForMaskedLM.from_pretrained(model_path, from_tf=bool(".ckpt" in model_path)) 

model.to(device)
model.eval() # 평가 

../../data11/model/distilbert/bert-re-kowiki-mecab/ is_fast:True


DistilBertForMaskedLM(
  (activation): GELUActivation()
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(139547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, in

In [4]:
#==================================================================================================
# load_dataset을 이용하여, 훈련/평가 dataset 로딩.
#
# [로컬 데이터 파일 로딩]
# => dataset = load_dataset("text", data_files='로컬.txt')       # text 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.csv')        # csv 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.tsv', delimiter="\t")  # tsv 로컬 파일 로딩
# => dataset = load_dataset("json", data_files='로컬.json')      # json 로컬 파일 로딩
# => dataset = load_dataset("pandas", data_files='로컬.pkl')     # pickled dataframe 로컬 파일 로딩
#
# [원격 데이터 파일 로딩]
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#    "train": url + "SQuAD_it-train.json.gz",
#    "test": url + "SQuAD_it-test.json.gz",
# }
# squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
#
# 출처 : https://wikidocs.net/166816
#==================================================================================================

from datasets import load_dataset

if is_Localfile == False:
    # 평가 말뭉치 로딩
    eval_dataset1 = load_dataset(eval_corpus)
    eval_dataset = eval_dataset1['test']
else:
    # 파일 평가 말뭉치 로딩
    eval_dataset1 = load_dataset("text", data_files=eval_corpus)
    eval_dataset = eval_dataset1['train']

# eval_dataset 출력해봄
print(f"eval_dataset========================================")
print(eval_dataset)
print(eval_dataset[0:3])

Using custom data configuration bongsoo--bongeval-c48674263e4810f8
Reusing dataset text (/MOCOMSYS/.cache/huggingface/datasets/text/bongsoo--bongeval-c48674263e4810f8/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


  0%|          | 0/1 [00:00<?, ?it/s]

eval_dataset========================================
Dataset({
    features: ['text'],
    num_rows: 1500
})
{'text': ['공공지원 민간임대주택은 주변 시세의 95  이하의 저렴한 임대료 혜택 외에도 품질좋은 주택에서 8년간 안심하고 살 수 있는 장점이 있다 ', '위기아동 발생을 미리 예방할 수 있는 지원활동이나 인프라 구축보다는 피해아동이 발생했을 때 그때그때 대처하는 방식에서 벗어나지 못하고 있다는 것이다 ', '여야가 각종 현안마다 정치적 타협을 모색하는 대신 검찰과 법원을 찾아 상대를 처벌해 달라고 독촉하는  정치의 사법화 가 극심해지고 있다는 지적이다 ']}


In [5]:
# tokenizer 처리
def tokenizer_function(examples):
    result =  tokenizer(examples['text'], truncation=True, max_length=token_max_len, return_overflowing_tokens=True)
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

%time eval_dataset_fast = eval_dataset.map(tokenizer_function, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

CPU times: user 749 ms, sys: 400 ms, total: 1.15 s
Wall time: 182 ms


In [6]:
print(eval_dataset_fast)
print(f'*fast_len:{len(eval_dataset_fast["text"])}, len:{len(eval_dataset["text"])}')  # fast_dataset과 dataset 길이를 비교함
print(eval_dataset_fast['text'][0:2])

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 1500
})
*fast_len:1500, len:1500
['공공지원 민간임대주택은 주변 시세의 95  이하의 저렴한 임대료 혜택 외에도 품질좋은 주택에서 8년간 안심하고 살 수 있는 장점이 있다 ', '위기아동 발생을 미리 예방할 수 있는 지원활동이나 인프라 구축보다는 피해아동이 발생했을 때 그때그때 대처하는 방식에서 벗어나지 못하고 있다는 것이다 ']


In [7]:
# MLM을 위한 DataCollatorForLangunageModeling 호출
from transformers import DataCollatorForLanguageModeling

# input_ids에 대해 MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# input_ids MLM 만들고 출력 해봄
mlm_eval_sample = data_collator(eval_dataset_fast['input_ids'][0:2])

print(f"eval_dataset_fast(MLM)=======================================")
print(mlm_eval_sample['input_ids'][0])
print(eval_dataset_fast['text'][0])

eval_dataset_fast(MLM)=======================================
tensor([   101, 120817,  12508,  14279, 120709,  36240,  14423,  16323, 119342,
         10892,    103, 138198,    103,  11978, 120705,  10459, 126430,  11102,
        121913,  38688, 125754,   9597,  35979, 123777, 119214,  10892, 121285,
         11489,    129,  10954,  18784, 132625,  12453,   9408,   9460,  13767,
        123073,  10739,  11506,    102,      0])
공공지원 민간임대주택은 주변 시세의 95  이하의 저렴한 임대료 혜택 외에도 품질좋은 주택에서 8년간 안심하고 살 수 있는 장점이 있다 


In [8]:
# 테스트 trainer 설정 
# trainer 

from transformers import Trainer, TrainingArguments

#========================================================================================
# 아래 computer_metrics 해도 안해도 동일한 loss 얻어옴
# => trainer = Trainer(compute_metrics=compute_metrics) 해줘야 함.

from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}
#========================================================================================

#########################################################################################
# hyper parameter 설정
#########################################################################################

eval_steps=int(len(eval_dataset_fast["text"]))           # 평가 스탭수
logging_steps=eval_steps                           # 로깅 스탭수(*평가스탭수 출력할때는 평가스탭수와 동일하게)

print(f'*total_optim_steps: {len(eval_dataset_fast["text"])}, *eval_steps:{eval_steps}, *logging_steps:{logging_steps}')
#########################################################################################

# cpu 사용이면 'no_cuda = True' 설정함.
no_cuda = False
if device == 'cpu':
    no_cuda = True
print(f'*no_cuda: {no_cuda}')

training_args = TrainingArguments(
    no_cuda = no_cuda,                       # GPU 사용  안함
    output_dir = OUTPATH,                   # 출력 모델 저장 경로 
    seed=333,
    per_gpu_train_batch_size=batch_size,    # 배치 사이즈 
    #prediction_loss_only=True,
    evaluation_strategy="steps",            # 평가 전략 : steps
    eval_steps=eval_steps,                  # 평가할 스텝수
    logging_steps=logging_steps             # 로깅할 스탭수
)

# trainer로 훈련할때는 [mask] 처리된 input_ids 만 dataset으로 넘겨주면 됨.
eval_dataset_fast_input_ids = eval_dataset_fast['input_ids']

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #MLM(Masked Language Model)
    eval_dataset=eval_dataset_fast_input_ids,      # 평가 데이터셋
    #compute_metrics=compute_metrics,
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


*total_optim_steps: 1500, *eval_steps:1500, *logging_steps:1500
*no_cuda: False


In [9]:
# 평가 
metrics = trainer.evaluate()
logger.info('{}'.format(metrics))
            

***** Running Evaluation *****
  Num examples = 1500
  Batch size = 8


2022-10-21 08:46:22,021 - distilbert-MLM-Test-Trainer - INFO - {'eval_loss': 2.6802070140838623, 'eval_runtime': 3.371, 'eval_samples_per_second': 444.977, 'eval_steps_per_second': 55.77}
